### De backtest class
Heeft een symbol van het aandeel om de data te vinden, maakt daar een dataframe van (tabel). Heeft verschillende methods met toepassing op de strategie/backtest, de eerste om de benodigde indicators te berekenen, de tweede method is om de koop/verkoop signalen te genereren door middel van masks die op hun beurt weer gebaseerd zijn op de verschillende indicatoren. Deze signalen genereren markeringen waarbij het signaal ofwel koop ofwel verkoop is, in de derde method worden dese koop en verkoop signalen met elkaar gekoppeld zodat er geen dubbele posities. Als er dus twee verkoop signalen achter elkaar zijn dan vervalt er dus 1. Als de strategie alleen long posities aangaat en het eerste signaal is een verkoop signaal waarna een koop signaal volgt dan vervalt het eerste signaal ook. De laatste method berekent de winst en andere benodigde gegevens om de prestaties van het algoritme te bepalen. Met deze backtest class kunnen ook strategieën die niet gebouwd zijn op technische indicatoren getest worden, de method voor het berekenen van de indicatoren en het bepalen van de signalen is dan anders.


In [1]:
import pandas as pd
import numpy as np
import ta
import matplotlib.pyplot as plt
from tiingo import TiingoClient

De numpy library zorgt ervoor dat we array's kunnen maken in plaats van python lists, deze nemen een stuk meer opslagruimte op omdat er allerlei informatie wordt opgeslagen die niet nodig is voor een lijst van getallen. Ook kan je met de numpy library goed met getallen werken. De pandas library is een van de belangrijkste bij het werken met veel data, het is de meest gebruikte library bij het werken met veel data in python. De ta library helpt ons met het berekenen van technische indicatoren, dit kunnen we zelf doen maar dit scheelt het invoeren van veel formules. Met de matplotlib library kunnen we mooi gegevens uit de data plotten. De laatste library die we gebruiken is de tiingo library, waarmee we de api van de historische data kunnen bedienen.

In [2]:
config = {}
config['session'] = True
config['api_key'] = "635bdbd888f766a407a43a68f241602aa5b218b9"

client = TiingoClient(config)

In de bovenstaande code verbinden we met de api via onze api key. Vanaf nu kunnen we met de client (de verbinding met de api) met historische data werken.

In [103]:
class Backtest:
    
    def __init__(self, symbol):
        self.symbol = symbol
        data = client.get_ticker_price(symbol,
                                            fmt='json',
                                            startDate='2010-01-01',
                                            endDate='2022-12-1',
                                            frequency='daily')
        self.df = pd.DataFrame(data)
        if self.df.empty:
            print("no data pulled")
        else:
            self.df['date'] = pd.to_datetime(self.df['date'])
            self.df.set_index('date', inplace=True)
            self.calc_indicators()
            self.generate_signals()
            self.match_signals()

        
        
    def calc_indicators(self):
        self.df['ma_20'] = self.df.close.rolling(20).mean()
        self.df['volatility'] = self.df.close.rolling(20).std()
        self.df['upper_bb'] = self.df.ma_20 + (2 * self.df.volatility)
        self.df['lower_bb'] = self.df.ma_20 - (2 * self.df.volatility)
        self.df['rsi'] = ta.momentum.rsi(self.df.close, window=6)
        self.df.dropna(inplace=True)
    
    def generate_signals(self):
        conditions = [(self.df.rsi < 50) & (self.df.close < self.df.lower_bb), (self.df.rsi > 50) & (self.df.close > self.df.upper_bb)]
        choices = ['Buy', 'Sell']
        self.df['signal'] = np.select(conditions, choices)
        self.df.signal = self.df.signal.shift()
        self.df.dropna(inplace=True)
    
    def match_signals(self):
        position = False
        buydates, selldates = [],[]
        
        for index, row in self.df.iterrows():
            if not position and row['signal'] == 'Buy':
                position = True
                buydates.append(index)
            
            if position and row['signal'] == 'Sell':
                position == False
                selldates.append(index)
        
        self.buy_array = self.df.loc[buydates].open
        self.sell_array = self.df.loc[selldates].open
        
    # def calc_profit(self):

In [104]:
apple = Backtest("AAPL")

In [107]:
apple.df

,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor,ma_20,volatility,upper_bb,lower_bb,rsi,signal
date,,,,,,,,,,,,,,,,,,
2010-02-02 00:00:00+00:00,195.86,196.32,193.380,195.910,24940800,5.961935,5.975937,5.886444,5.963457,698343098,0.0,1.0,206.15895,6.776343,219.711637,192.606263,36.993595,0
2010-02-03 00:00:00+00:00,199.23,200.20,194.420,195.170,21976000,6.064517,6.094043,5.918101,5.940931,615328615,0.0,1.0,205.40145,6.654663,218.710777,192.092123,45.925378,0
2010-02-04 00:00:00+00:00,192.05,198.37,191.570,196.730,27059000,5.845959,6.038339,5.831348,5.988417,757652757,0.0,1.0,204.45545,7.147913,218.751275,190.159625,33.708326,0
2010-02-05 00:00:00+00:00,195.46,196.00,190.850,192.625,30368100,5.949759,5.966196,5.809431,5.863462,850307650,0.0,1.0,203.69945,7.264687,218.228823,189.170077,42.435604,0
2010-02-08 00:00:00+00:00,194.12,197.88,194.000,195.690,17081100,5.908969,6.023423,5.905317,5.956760,478271278,0.0,1.0,202.80645,7.290900,217.388249,188.224651,39.955187,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-25 00:00:00+00:00,148.11,148.88,147.120,148.305,35195860,148.110000,148.880000,147.120000,148.305000,35195860,0.0,1.0,146.91850,5.735985,158.390469,135.446531,48.288239,0
2022-11-28 00:00:00+00:00,144.22,146.64,143.380,145.140,69346522,144.220000,146.640000,143.380000,145.140000,69346522,0.0,1.0,146.34250,5.370271,157.083042,135.601958,36.444039,0
2022-11-29 00:00:00+00:00,141.17,144.81,140.355,144.290,83763803,141.170000,144.810000,140.355000,144.290000,83763803,0.0,1.0,145.73400,5.223130,156.180260,135.287740,29.610552,0


In [109]:
position = False
buydates, selldates = [],[]

for index, row in apple.df.iterrows():
    if not position and row['signal'] == 'Buy':
        position = True
        buydates.append(index)

    if position and row['signal'] == 'Sell':
        position == False
        selldates.append(index)

buy_array = apple.df.loc[buydates].open
sell_array = apple.df.loc[selldates].open

print(buydates)

[Timestamp('2010-08-25 00:00:00+0000', tz='UTC')]
